# JetBot - Multi-Task Learning

Before Starting this task, you need to complete following two tasks:
1. Collect Data for Training multiple tasks, please use data_collection_multi_task.ipynb
2. Train Model for multi-task learning - please use multi_task_training_model.ipynb

We will use trained model "best_model_adam.pth" for inferernces

## On JetBot

Import all required libraries for Camera and Motor Interfacing, ResNet18 Inferences using Torch Libraries

In [1]:
import torch
import torchvision
import torch.nn.functional as F

In [2]:
device = torch.device('cuda')

In [3]:
model = torchvision.models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 7)
model.load_state_dict(torch.load('best_model_adam.pth'))
model = model.eval()
model = model.to(device).half()

Let's display the robots live video feed.  We won't need fast framerates for this, so we'll lower the capturing framerate to 10FPS to reduce streaming bandwidth.

In [4]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot.camera.opencv_gst_camera import OpencvGstCamera
from jetbot.image import bgr8_to_jpeg

camera = OpencvGstCamera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

turn_slider = widgets.FloatSlider(descrption='turn', min=-1.0, max=1.0, step=0.01, orientation='horizontal')

blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
blocked_stop = widgets.FloatSlider(description='stop', min=0.0, max=1.0, orientation='vertical')


display(widgets.HBox([image, blocked_slider, blocked_stop, turn_slider]))

Following pre-processing is performed to normalize and convert to RGB incoming image

In [5]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x.half()

In a seperate thread, we'll continuously run the neural network on the camera feed.

In [6]:
from jetbot.robot.basic_jetbot import BasicJetbot

robot = BasicJetbot()

In [7]:
def get_steering(y):
    z = y.cpu().float().flatten()[0:3]
    return torch.sum(z * torch.Tensor([-1.0, 0.0, 1.0])) / z.sum()

Control Options for Speed, Steeering Gain, and IIR Filter 

In [8]:
speed_slider = widgets.FloatSlider(description='speed', min=0.0, max=1.0, step=0.01, value=0.22)
steering_gain_slider = widgets.FloatSlider(description='gain', min = 0.0, max=1.0, step=0.01, value=0.20)
iir_filter_slider = widgets.FloatSlider(description='iir', min=0.0, max=1.0, step=0.01, value=0.15)
display(speed_slider)
display(steering_gain_slider)
display(iir_filter_slider)

FloatSlider(value=0.22, description='speed', max=1.0, step=0.01)

FloatSlider(value=0.2, description='gain', max=1.0, step=0.01)

FloatSlider(value=0.15, description='iir', max=1.0, step=0.01)

Following Slider will help you to control Stopping time at Intersection

In [9]:
style = {'description_width': 'initial'}
stopping_time_slider = widgets.FloatSlider(description='stopping time', min=0.0, max=5.0, step=0.1, value=1.0, style = style)
display(stopping_time_slider)

FloatSlider(value=1.0, description='stopping time', max=5.0, style=SliderStyle(description_width='initial'))

Control Options to Set JetBot Directions

In [10]:
switch_slider = widgets.ToggleButtons(
    options=['Right', 'Forward', 'Left'],
    description='JetBot Direction:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
    style = style,
    value = 'Forward'
)
display(switch_slider)

ToggleButtons(description='JetBot Direction:', index=1, options=('Right', 'Forward', 'Left'), style=ToggleButt…

Following code snippet will help you control robot states. We currently have 4 robot states mainly for Follow road, Stop JetBot, and for turning JetBot Right or Left.

In [23]:
# Import State Machine Library to control robot states
# To install State Machine Library: $python3 -m pip install python-statemachine
from statemachine import StateMachine, State
import time

class RobotControl(StateMachine):
    RunJetbot = State ('run_robot', initial = True)
    StopJetbot = State ('stop_robot')
    TurnRightJetbot = State ('turn_right_robot')
    TurnLeftJetbot = State ('turn_left_robot')
    
    slowdown = RunJetbot.to(StopJetbot) | TurnRightJetbot.to(StopJetbot) | TurnLeftJetbot.to(StopJetbot) | StopJetbot.to.itself()
    go = StopJetbot.to(RunJetbot) | TurnRightJetbot.to(RunJetbot) | TurnLeftJetbot.to(RunJetbot)| RunJetbot.to.itself()
    goRight = StopJetbot.to(TurnRightJetbot)
    goLeft = StopJetbot.to(TurnLeftJetbot)
    
    global steering    
    def on_slowdown(self):
        robot.stop()
        time.sleep(stopping_time_slider.value)
    def on_goRight(self):
        robot.forward(speed_slider.value)
        time.sleep(2.3)
        robot.right(speed_slider.value)
        time.sleep(1.25)
    def on_goLeft(self):
        robot.forward(speed_slider.value)
        time.sleep(2.3)
        robot.left(speed_slider.value)
        time.sleep(1.55)
    def on_go(self, value):
        global steering_last
        steering_last = steering_last
        steering = float(get_steering(value)) * float(1.0 - iir_filter_slider.value) + float(steering_last * iir_filter_slider.value)
        turn_slider.value = steering
        steering_last = steering
        robot.set_motors(max(speed_slider.value + steering_gain_slider.value * steering, 0), 
                     max(speed_slider.value  - steering_gain_slider.value * steering, 0)
        )
    

We will initialize inferences here to control the robot states.
1. STOP: If Probility of Collison (prob_collision) and Probibilty of Crosswalk Detection (prob_crosswalk) is HIGH (> 0.65)
2. RUN: If Probility of Collison (prob_collision) and Probibilty of Crosswalk Detection (prob_crosswalk) is LOW (< 0.65)
    2A. Turn Right: If Switch Slider is on Right
    2B. Turn Left: If Switch Slider is on Left

In [24]:
robot_control = RobotControl()
intersection_count = 0
def execute_model(change):
    global robot, turn_slider, model, iir_filter_slider, turn_slider, speed_slider
    global intersection_count
    x = change['new']
    t1 = time.time()
    x = preprocess(x)
    y= model(x)
    #steering = get_steering(y) * (1.0 - iir_filter_slider.value) + steering_last * iir_filter_slider.value
    y_collision = F.softmax(y, dim=1)
    prob_collision = float(y_collision.flatten()[3])    
    blocked_slider.value = prob_collision
    
    prob_crosswalk = float(y_collision.flatten()[5])
    blocked_stop.value = prob_crosswalk
    
    if prob_collision < 0.65 :
        if prob_crosswalk < 0.8:
            intersection_count = 0
            robot_control.go(y)
        else:
            if (intersection_count == 0 and switch_slider.value == 'Forward'):
                robot_control.slowdown()
                intersection_count = 1
            if (intersection_count == 0 and switch_slider.value == 'Right'):
                robot_control.slowdown()
                robot_control.goRight()
                robot_control.go(y)
                intersection_count = 1
            if (intersection_count == 0 and switch_slider.value == 'Left'):
                robot_control.slowdown()
                robot_control.goLeft()
                robot_control.go(y)
                intersection_count = 1
            else:
                robot_control.go(y)  
    else:
        robot_control.slowdown()
    
    time.sleep(0.001)
    
execute_model({'new': camera.value})
robot.stop()

In [25]:
camera.observe(execute_model, names='value')  # call this to attach callback

In [ ]:
camera.unobserve(execute_model, names='value')  # call this to un-attach callback

In [26]:
camera.unobserve_all()

In [27]:
robot.stop()